# 1. Install and Import Dependencies

In [2]:
!pip install mediapipe opencv-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip


In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import time

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [23]:
cap = cv2.VideoCapture(0)
hands_list = []

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
    
        #image stuff
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_height, image_width, _ = image.shape
        image = cv2.flip(image, 1)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Rendering results
        if results.multi_hand_landmarks:
            for handLandmarks in results.multi_hand_landmarks:
                hand_dict = {}
                for point in mp_hands.HandLandmark:
                    normalizedLandmark = handLandmarks.landmark[point]
                    pixelCoordinatesLandmark = mp_drawing._normalized_to_pixel_coordinates(normalizedLandmark.x, normalizedLandmark.y, image_width, image_height)
                    hand_dict[point.name] = pixelCoordinatesLandmark
                hands_list.append(hand_dict)

            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                    mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                )
            
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('s'):
            timestamp = time.strftime("%Y%m%d-%H%M%S")
            filename = f"HandTracking_{timestamp}.png"
            cv2.imwrite("saved_imgs/" + filename, image)
            print(f"Image saved: {filename}")

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

HandLandmark.WRIST
None
HandLandmark.THUMB_CMC
(404, 1036)
HandLandmark.THUMB_MCP
(345, 865)
HandLandmark.THUMB_IP
(309, 716)
HandLandmark.THUMB_TIP
(243, 611)
HandLandmark.INDEX_FINGER_MCP
(549, 712)
HandLandmark.INDEX_FINGER_PIP
(531, 530)
HandLandmark.INDEX_FINGER_DIP
(525, 413)
HandLandmark.INDEX_FINGER_TIP
(527, 317)
HandLandmark.MIDDLE_FINGER_MCP
(677, 729)
HandLandmark.MIDDLE_FINGER_PIP
(724, 532)
HandLandmark.MIDDLE_FINGER_DIP
(755, 417)
HandLandmark.MIDDLE_FINGER_TIP
(783, 321)
HandLandmark.RING_FINGER_MCP
(784, 795)
HandLandmark.RING_FINGER_PIP
(865, 616)
HandLandmark.RING_FINGER_DIP
(922, 506)
HandLandmark.RING_FINGER_TIP
(975, 407)
HandLandmark.PINKY_MCP
(865, 900)
HandLandmark.PINKY_PIP
(960, 803)
HandLandmark.PINKY_DIP
(1018, 742)
HandLandmark.PINKY_TIP
(1073, 681)
HandLandmark.WRIST
(922, 1039)
HandLandmark.THUMB_CMC
(767, 916)
HandLandmark.THUMB_MCP
(693, 730)
HandLandmark.THUMB_IP
(674, 577)
HandLandmark.THUMB_TIP
(614, 468)
HandLandmark.INDEX_FINGER_MCP
(859, 553)
Han

In [26]:
for val in hands_list:
    print(val)

{'WRIST': None, 'THUMB_CMC': (404, 1036), 'THUMB_MCP': (345, 865), 'THUMB_IP': (309, 716), 'THUMB_TIP': (243, 611), 'INDEX_FINGER_MCP': (549, 712), 'INDEX_FINGER_PIP': (531, 530), 'INDEX_FINGER_DIP': (525, 413), 'INDEX_FINGER_TIP': (527, 317), 'MIDDLE_FINGER_MCP': (677, 729), 'MIDDLE_FINGER_PIP': (724, 532), 'MIDDLE_FINGER_DIP': (755, 417), 'MIDDLE_FINGER_TIP': (783, 321), 'RING_FINGER_MCP': (784, 795), 'RING_FINGER_PIP': (865, 616), 'RING_FINGER_DIP': (922, 506), 'RING_FINGER_TIP': (975, 407), 'PINKY_MCP': (865, 900), 'PINKY_PIP': (960, 803), 'PINKY_DIP': (1018, 742), 'PINKY_TIP': (1073, 681)}
{'WRIST': (922, 1039), 'THUMB_CMC': (767, 916), 'THUMB_MCP': (693, 730), 'THUMB_IP': (674, 577), 'THUMB_TIP': (614, 468), 'INDEX_FINGER_MCP': (859, 553), 'INDEX_FINGER_PIP': (836, 349), 'INDEX_FINGER_DIP': (831, 220), 'INDEX_FINGER_TIP': (834, 109), 'MIDDLE_FINGER_MCP': (990, 563), 'MIDDLE_FINGER_PIP': (1033, 344), 'MIDDLE_FINGER_DIP': (1066, 206), 'MIDDLE_FINGER_TIP': (1099, 89), 'RING_FINGER_M